# Graph Sage sử dụng cho Node representation

## Dataset

Protein protein interaction

### Input format <br/>

 File-G.json --  json file chứa thông tin đầu vào của graph về node, edge. <br/>
 File-id_map.json -- Lưu thông tin về id của các node. <br/>
 File-class_map.json -- Lưu thông tin về class hay label của các node. <br/>
 File-feats.npy [optional] --- Node feature, node attribute <br/>
 File-walks.txt [optional] --- file chỉ định các lần xuất hiện random walk.

## Embedding

Chạy lệnh:<br/>
python -m graphsage.unsupervised_train --train_prefix ./ppi_data/ppi --model graphsage_mean --max_total_steps 1000 --validate_iter 10

thông tin embedd của các node sẽ được lưu lại trong file val.npy

# Node classification

In [2]:
import numpy as np
from networkx.readwrite import json_graph
import json

dataset_dir = './GraphSAGE/ppi_data'
data_dir = './GraphSAGE/unsup-ppi_data/graphsage_mean_small_0.000010'

### Xử lí data

In [3]:
G = json_graph.node_link_graph(json.load(open(dataset_dir + "/ppi-G.json")))
labels = json.load(open(dataset_dir + "/ppi-class_map.json"))
labels = {int(i):l for i, l in labels.items()}

train_ids = [n for n in G.nodes() if not G.node[n]['val'] and not G.node[n]['test']]
train_labels = np.array([labels[i] for i in train_ids])
test_ids = [n for n in G.nodes() if G.node[n]['val'] or G.node[n]['test']]
test_labels = np.array([labels[i] for i in test_ids])

embeds = np.load(data_dir + "/val.npy")

id_map = {}
with open(data_dir + "/val.txt") as fp:
    for i, line in enumerate(fp):
        id_map[int(line.strip())] = i
train_embeds = embeds[[id_map[id] for id in train_ids]]
test_embeds = embeds[[id_map[id] for id in test_ids]]

### vector node embedding

In [4]:
print("Train data: %s" %len(train_embeds))
print("Test data: %s" %len(test_embeds))

# vector của node 0 
print(train_embeds[0])

Train data: 44906
Test data: 12038
[-0.05542022  0.03955797 -0.03278705  0.013212   -0.07131557 -0.02094384
 -0.03544485  0.06932638 -0.04968539  0.07765641  0.1317103   0.18846025
  0.0693346  -0.05343338  0.05961615  0.02985436  0.03919346  0.09237635
  0.03041657 -0.01732718  0.04885008 -0.0255577  -0.00207216 -0.00488547
  0.00750719  0.05839671  0.05775898  0.07103132  0.02165969 -0.12403699
  0.04033683  0.0477627  -0.23823944 -0.09042604 -0.03936075 -0.05098277
 -0.04373214  0.08860636  0.02481151 -0.00338339 -0.05418244  0.072975
 -0.05043067  0.02901203 -0.00058473 -0.05670315 -0.01538582 -0.11889282
  0.03093199  0.04993587  0.03645257  0.0606937  -0.06639259  0.09447865
  0.00764183 -0.03181717  0.01144708 -0.09786566  0.02439624 -0.03185292
  0.06483772 -0.04504294 -0.01713146 -0.03242396  0.03894665  0.03075094
 -0.03840124 -0.01867723 -0.02584256  0.01045557 -0.00799712  0.04068759
 -0.02996851 -0.01341127  0.06472888 -0.14384094  0.07961451  0.0700298
 -0.03414496  0.062

In [5]:
print(train_embeds.shape)

(44906, 256)


### model

Do mỗi node có đến 121 label nên chọn ra 1 label để classifier

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log = LogisticRegression()
id = np.random.randint(0,121)
log.fit(train_embeds, train_labels[:, id])

print("accuracy score:", accuracy_score(test_labels[:, id], log.predict(test_embeds)))

c:\users\dell\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy score: 0.7492108323641801
